<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [111]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


And then we import our movie database

In [112]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [113]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

In [114]:
%%sql
INSERT INTO teathers(name, capacity)
VALUES ("Lund", 200),
       ("Malmö", 400)

 * sqlite:///movies.sqlite
2 rows affected.


[]

In [115]:
%%sql
INSERT INTO movies(imdb_key, title, prod_year, running_time)
VALUES ("tgrjgu64737", "Stab Reveal 2022", "2022", 5),
       ("tgdfgdfg75", "12 Mighty Orphans", "2021", 118)

 * sqlite:///movies.sqlite
2 rows affected.


[]

In [116]:
%%sql
INSERT INTO screenings(uid, start_date, start_time, title, teather, seats, imdb_key)
VALUES ("abc123", "2022-01-01", "19:30", "Stab Reveal 2022", "Lund", 200, "tgrjgu64737"),
       ("123abc", "2022-02-21", "20:00", "12 Mighty Orphans", "Malmö", 400, "tgdfgdfg75")

 * sqlite:///movies.sqlite
2 rows affected.


[]

In [117]:
%%sql
INSERT INTO customers(username, full_name, password)
VALUES ("an8218fr-s", "André Frisk", "ajabaja"),
       ("lucat", "Lina Ekskog", "abc123")

 * sqlite:///movies.sqlite
2 rows affected.


[]

In [118]:
%%sql
INSERT INTO ticket(screening, ticket_holder)
VALUES ("abc123", "an8218fr-s"),
       ("123abc", "lucat")

 * sqlite:///movies.sqlite
2 rows affected.


[]

## SQL statements to review at the lab session

Write SQL code for the following tasks:

+ Show the names of all movies.

In [119]:
%%sql
SELECT *
FROM movies

 * sqlite:///movies.sqlite
Done.


imdb_key,title,prod_year,running_time
tgrjgu64737,Stab Reveal 2022,2022,5
tgdfgdfg75,12 Mighty Orphans,2021,118


+ Show the performance dates for one of the movies.

In [120]:
%%sql
SELECT *
FROM screenings

 * sqlite:///movies.sqlite
Done.


uid,start_date,start_time,title,teather,seats,imdb_key
abc123,2022-01-01,19:30,Stab Reveal 2022,Lund,200,tgrjgu64737
123abc,2022-02-21,20:00,12 Mighty Orphans,Malmö,400,tgdfgdfg75


+ Show all data concerning performances at a given theatere
  on a given date.

In [121]:
%%sql
SELECT *
FROM teathers AS t
JOIN screenings AS s
ON s.teather = t.name
WHERE t.name = "Lund" AND s.start_date = "2022-01-01"

 * sqlite:///movies.sqlite
Done.


name,capacity,uid,start_date,start_time,title,teather,seats,imdb_key
Lund,200,abc123,2022-01-01,19:30,Stab Reveal 2022,Lund,200,tgrjgu64737


+ List all customers

In [122]:
%%sql
SELECT *
FROM customers

 * sqlite:///movies.sqlite
Done.


username,full_name,password
an8218fr-s,André Frisk,ajabaja
lucat,Lina Ekskog,abc123


+ List all tickets

In [123]:
%%sql
SELECT *
FROM ticket

 * sqlite:///movies.sqlite
Done.


id,screening,ticket_holder
1842530da30cb67b1456d20f67f09fe7,abc123,an8218fr-s
544d3f743cd236d0fa45a0ad061ea851,123abc,lucat


## Things to try out yourselves (not reviewed at the lab session)

During lab 3 you'll write a REST service for the database
you just created, and for the server to work properly, the
things below should work.

If you have any questions about the material below, feel
free to ask them at the QA sessions (see Moodle for how to
sign up for them).


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [124]:
%%sql
INSERT 
INTO ticket(screening, ticket_holder)
VALUES ("123abc", "lucat")

 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [125]:
%%sql
SELECT *
FROM ticket

 * sqlite:///movies.sqlite
Done.


id,screening,ticket_holder
1842530da30cb67b1456d20f67f09fe7,abc123,an8218fr-s
544d3f743cd236d0fa45a0ad061ea851,123abc,lucat
1efe486c00f68dc9008a53cdf03f26ea,123abc,lucat


+ Try to insert two movie theaters with the same name (this
  should fail).

In [126]:
%%sql
INSERT 
INTO teathers(name, capacity)
VALUES ("Helsingborg", 50), ("Lomma", 75)

 * sqlite:///movies.sqlite
2 rows affected.


[]

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [127]:
%%sql
INSERT 
INTO screenings(start_time, title, teather)
VALUES ("1998-09-18", "Stab Reveal 2022", "Tomelilla")

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT 
INTO screenings(start_time, title, teather)
VALUES ("1998-09-18", "Stab Reveal 2022", "Tomelilla")]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


+ Create a ticket where either the user or the performance
  doesn’t exist (this should fail).

In [128]:
%%sql
INSERT 
INTO ticket(screening, ticket_holder)
VALUES ("123abc", "sebbe")

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT 
INTO ticket(screening, ticket_holder)
VALUES ("123abc", "sebbe")]
(Background on this error at: https://sqlalche.me/e/14/gkpj)
